In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dukto.pipe import Pipe
from dukto.processor import ColProcessor, MultiColProcessor, Transformer
import pandas as pd
import numpy as np
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer

In [3]:
data  = pd.read_csv('data/ufc.csv', index_col=0)

# ColProcessor
### applies function/s to a column/s  

In [4]:
def convert_foot_to_cm(r):
    if isinstance(r, str) and "'" in r:
        foot, inches = r.split("'")
        inches = int(foot)*12 + int(inches.replace('"', ''))
        return inches*2.54
    return np.nan

def convert_inch_to_cm(r):
    if isinstance(r,str) and '"' in r:
        return int(r.replace('"', '')) * 2.54
    return np.nan

def num_of_num_to_perc(r):
    if isinstance(r,str) and 'of' in r:
        thr, landed = map(int, r.split('of'))
        if landed > 0:
            return thr / landed 
    return np.nan

def pounds_to_kg(r):
    if isinstance(r, str) and 'lbs' in r:
        return int(r.split(' ')[0]) * 0.4535
    return r

In [5]:
single_pipe = [
    ColProcessor(name=['agg_height_first','agg_height_second'], 
                 funcs=[convert_foot_to_cm], funcs_test={"6'2\"":187.96}, suffix='_new'),
    
    ColProcessor(name=['agg_reach_first','agg_reach_second'], 
                 funcs=[convert_inch_to_cm], funcs_test={'70"': 177.80}, suffix='_new'),
    
    ColProcessor(name=['second_total_str', 'first_total_str'], 
                 funcs=[num_of_num_to_perc], suffix='_%%_new', funcs_test={'50 of 100':0.5}),
    
    ColProcessor(name=['agg_dob_first', 'agg_dob_second', 'date_card'], 
                 funcs=[pd.to_datetime]),
    
    ColProcessor(name='agg_weight_first', new_name={"agg_weight_first":'weight_class'}, 
                 funcs=[pounds_to_kg], suffix='_new', drop=True)
]

## MultiColProcessor

## applies a function that takes and returns a pandas DataFrame
## this class is used to add columns based on other column/s

In [28]:
def add_ages(df):
    df['first_fighter_age_new'] = df['date_card'] - df['agg_dob_first']
    df['second_fighter_age_new'] = df['date_card'] - df['agg_dob_second']
    return df

def ages_in_years(df):
    df[['first_fighter_age_new', 'second_fighter_age_new']] = df[['first_fighter_age_new', 'second_fighter_age_new']].applymap(lambda x:x/np.timedelta64(1, 'Y'))
    return df

In [29]:
multi_pipe = [
    MultiColProcessor(name=['first_fighter_age_new', 'second_fighter_age_new'], 
                      funcs=[add_ages, ages_in_years]),
             ]

## Transformer

### applies a feature_engine style transformer to a column/s

In [36]:

new_cols_func = lambda x: [i for i in x if (('new' in i) and ('weight' not in i))]

trans_pipe  = [
    Transformer(name_from_func=new_cols_func, 
                transformers=[MeanMedianImputer], imputation_method='median', mode='transform'),
    
    MultiColProcessor(funcs=[lambda x:x.assign(weight_class_new=x.weight_class_new.astype(str))]),
    
    Transformer(name=['weight_class_new'], 
                transformers=[CategoricalImputer,CountFrequencyEncoder], mode='transform'),
]

In [37]:
pipeline = single_pipe+multi_pipe+trans_pipe

In [38]:
pipe = Pipe(data=data, pipeline=pipeline, run_test_cases=False, mode='fit_transform')

In [39]:
res = pipe.run()

Runningconvert_foot_to_cm(agg_height_first)
Runningconvert_foot_to_cm(agg_height_second)
Runningconvert_inch_to_cm(agg_reach_first)
Runningconvert_inch_to_cm(agg_reach_second)
Runningnum_of_num_to_perc(second_total_str)
Runningnum_of_num_to_perc(first_total_str)
Runningto_datetime(agg_dob_first)
Runningto_datetime(agg_dob_second)
Runningto_datetime(date_card)
Runningpounds_to_kg(agg_weight_first)
added columns ['second_fighter_age_new', 'first_fighter_age_new']... deleted columns []
added columns []... deleted columns []


In [40]:
pip2 = Pipe(data=data.head(20), pipeline=pipe.pipeline, run_test_cases=False, mode='transform')

In [41]:
pip2.run()

Runningconvert_foot_to_cm(agg_height_first)
Runningconvert_foot_to_cm(agg_height_second)
Runningconvert_inch_to_cm(agg_reach_first)
Runningconvert_inch_to_cm(agg_reach_second)
Runningnum_of_num_to_perc(second_total_str)
Runningnum_of_num_to_perc(first_total_str)
Runningto_datetime(agg_dob_first)
Runningto_datetime(agg_dob_second)
Runningto_datetime(date_card)
Runningpounds_to_kg(agg_weight_first)
added columns ['second_fighter_age_new', 'first_fighter_age_new']... deleted columns []
added columns []... deleted columns []


,agg_dob_first,agg_dob_second,agg_height_first,agg_height_second,agg_reach_first,agg_reach_second,agg_stand_first,agg_stand_second,agg_str_acc_first,agg_str_acc_second,...,type,agg_height_first_new,agg_height_second_new,agg_reach_first_new,agg_reach_second_new,second_total_str_%%_new,first_total_str_%%_new,weight_class_new,first_fighter_age_new,second_fighter_age_new
0,1987-07-19,1989-12-26,"6' 4""","6' 4""","84""","77""",Orthodox,Southpaw,57%,50%,...,belt,193.04,193.04,213.36,195.58,0.452471,0.629412,454,32.559190,30.119715
1,1988-03-07,1988-12-28,"5' 5""","5' 9""","66""","68""",Southpaw,Orthodox,51%,35%,...,belt,165.10,175.26,167.64,172.72,0.397059,0.695122,382,31.923996,31.113575
2,1992-01-16,1993-12-13,"6' 5""","6' 0""","80""","74""",Orthodox,Southpaw,55%,55%,...,NaN,195.58,182.88,203.20,187.96,0.666667,0.636364,96,28.063547,26.155226
3,1991-02-16,1991-08-06,"5' 8""","5' 7""","70""","71""",Orthodox,Orthodox,41%,46%,...,NaN,172.72,170.18,177.80,180.34,0.547009,0.391892,614,28.978008,28.509826
4,1985-02-07,1983-07-28,"6' 3""","5' 10""","79""","73""",Orthodox,Orthodox,50%,39%,...,NaN,190.50,177.80,200.66,185.42,0.805195,0.465517,31,35.001403,36.534631
5,1992-08-06,1986-06-04,"6' 0""","6' 2""","74""","73""",Orthodox,Orthodox,56%,49%,...,fight,182.88,187.96,187.96,185.42,0.602041,0.526946,687,27.507752,33.681732
6,1983-07-27,1989-02-11,"5' 5""","5' 6""","67""","69""",Orthodox,Orthodox,40%,50%,...,NaN,165.10,167.64,170.18,175.26,0.394834,0.469298,382,36.537369,30.990369
7,1990-08-16,1994-03-30,"5' 11""","6' 0""","72""","77""",Orthodox,Orthodox,42%,45%,...,perf,180.34,182.88,182.88,195.58,0.588235,0.250000,1012,29.481783,25.862270
8,1994-03-30,1993-07-01,"5' 7""","5' 9""","66""","69""",Orthodox,Switch,48%,45%,...,perf,170.18,175.26,167.64,175.26,0.268657,0.295455,626,25.862270,26.606980
9,1990-01-05,1989-03-07,"6' 0""","5' 5""","73""","67""",Southpaw,Orthodox,50%,32%,...,NaN,182.88,165.10,185.42,170.18,0.538462,0.250000,626,30.092336,30.924660


# after 

In [13]:
res.head(5)[[i for i in res.columns if 'new' in i]]

,agg_height_first_new,agg_height_second_new,agg_reach_first_new,agg_reach_second_new,second_total_str_%%_new,first_total_str_%%_new,weight_class_new,first_fighter_age_new,second_fighter_age_new
0,193.04,193.04,213.36,195.58,0.452471,0.629412,454,32.559190,30.119715
1,165.10,175.26,167.64,172.72,0.397059,0.695122,382,31.923996,31.113575
2,195.58,182.88,203.20,187.96,0.666667,0.636364,96,28.063547,26.155226
3,172.72,170.18,177.80,180.34,0.547009,0.391892,614,28.978008,28.509826
4,190.50,177.80,200.66,185.42,0.805195,0.465517,31,35.001403,36.534631


# Before

In [14]:
data.head(5)

,agg_dob_first,agg_dob_second,agg_height_first,agg_height_second,agg_reach_first,agg_reach_second,agg_stand_first,agg_stand_second,agg_str_acc_first,agg_str_acc_second,...,date_card,first_fighter_res,first_sig_str_,first_sig_str_percentage,first_total_str,method,second_sig_str_percentage,second_total_str,time,type
0,19-Jul-87,26-Dec-89,"6' 4""","6' 4""","84""","77""",Orthodox,Southpaw,57%,50%,...,8-Feb-20,W,104 of 166,62%,107 of 170,Decision - Unanimous,44%,119 of 263,5:00,belt
1,7-Mar-88,28-Dec-88,"5' 5""","5' 9""","66""","68""",Southpaw,Orthodox,51%,35%,...,8-Feb-20,W,40 of 65,61%,57 of 82,KO/TKO,30%,27 of 68,1:03,belt
2,16-Jan-92,13-Dec-93,"6' 5""","6' 0""","80""","74""",Orthodox,Southpaw,55%,55%,...,8-Feb-20,L,7 of 11,63%,7 of 11,KO/TKO,66%,10 of 15,1:59,NaN
3,16-Feb-91,6-Aug-91,"5' 8""","5' 7""","70""","71""",Orthodox,Orthodox,41%,46%,...,8-Feb-20,L,17 of 60,28%,29 of 74,Decision - Split,48%,64 of 117,5:00,NaN
4,7-Feb-85,28-Jul-83,"6' 3""","5' 10""","79""","73""",Orthodox,Orthodox,50%,39%,...,8-Feb-20,W,20 of 50,40%,27 of 58,Decision - Unanimous,41%,62 of 77,5:00,NaN


In [36]:
class ColdpRoasdf():
    pass

In [37]:
a = ColdpRoasdf()

In [38]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [39]:
str(a.__class__).split('.')[-1][:-2]

'ColdpRoasdf'

In [12]:
a = [1,2,3,4,5]

In [14]:
a[1] = 5

[1, 5, 3, 4, 5]